# Módulo 2: Scraping con Selenium
## LATAM Airlines
<a href="https://www.latam.com/es_ar/"><img src="https://i.pinimg.com/originals/dd/52/74/dd5274702d1382d696caeb6e0f6980c5.png"  width="420"></img></a>
<br>

Vamos a scrapear el sitio de Latam para averiguar datos de vuelos en funcion el origen y destino, fecha y cabina. La información que esperamos obtener de cada vuelo es:
- Precio(s) disponibles
- Horas de salida y llegada (duración)
- Información de las escalas

**¡Empecemos!**
Utilicemos lo aprendido hasta ahora para lograr el objetivo propuesto

## Selenium

Selenium es una herramienta que nos permitirá controlar un navegador y podremos utilizar las funcionalidades del motor de JavaScript para cargar el contenido que no viene en el HTML de la página. Para esto necesitamos el módulo `webdriver`.

In [1]:
from selenium import webdriver
#importampos libreria para cargar el driver automaticamente
from webdriver_manager.firefox import GeckoDriverManager

url='https://www.latamairlines.com/ar/es/ofertas-vuelos?origin=ASU&inbound=null&outbound=2022-12-01T15%3A00%3A00.000Z&destination=MAD&adt=1&chd=0&inf=0&trip=OW&cabin=Economy&redemption=false&sort=RECOMMENDED'

Necesitamos controladores web para diferentes navegadores web.<br>
Paso 1: instanciar un **driver** del navegador

In [2]:
options = webdriver.FirefoxOptions()
# Podemos agregarle opciones al driver para utilizar los distintos modos del navegador
options.add_argument('-private')
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install(), options=options)

[WDM] - Downloading: 19.0kB [00:00, 3.91MB/s]                   
[WDM] - Downloading: 19.0kB [00:00, 4.89MB/s]                   
[WDM] - Downloading: 100%|██████████| 1.58M/1.58M [00:00<00:00, 4.35MB/s]
C:\Users\mario\AppData\Local\Temp\ipykernel_17420\1250048064.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=GeckoDriverManager().install(), options=options)


Paso 2: hacer que el navegador cargue la página web.

In [3]:
driver.get(url)

paso 3:Extraer informacion de la pagina<br>
Carguemos la página y analicemos dónde se encuentra la información<br>
Vemos que el bloque de vuelos se encuentra en una `ul` y que cada vuelo es un item de la lista, `li`. <br>

In [4]:
#Usaremos el Xpath para obtener la lista de vuelos
vuelos = driver.find_elements('xpath','//ol/li')
print (vuelos)

[<selenium.webdriver.remote.webelement.WebElement (session="b25c4f0b-2236-45f6-ac4d-42119dd974a4", element="af6d0950-83d0-4bcc-a735-633437784cd4")>, <selenium.webdriver.remote.webelement.WebElement (session="b25c4f0b-2236-45f6-ac4d-42119dd974a4", element="41e623b0-61d3-483d-bbcf-d9bf082bdeb5")>, <selenium.webdriver.remote.webelement.WebElement (session="b25c4f0b-2236-45f6-ac4d-42119dd974a4", element="f89d7ae8-2571-4a85-8888-8327d294f613")>, <selenium.webdriver.remote.webelement.WebElement (session="b25c4f0b-2236-45f6-ac4d-42119dd974a4", element="607b1fe8-ad3b-4883-9323-846bca107791")>, <selenium.webdriver.remote.webelement.WebElement (session="b25c4f0b-2236-45f6-ac4d-42119dd974a4", element="10e8805f-7d05-4af0-9445-5b5154838328")>, <selenium.webdriver.remote.webelement.WebElement (session="b25c4f0b-2236-45f6-ac4d-42119dd974a4", element="fca891e6-503c-4e8c-b4ce-d851f6ddf9ef")>, <selenium.webdriver.remote.webelement.WebElement (session="b25c4f0b-2236-45f6-ac4d-42119dd974a4", element="d7e2

Obtengamos la informacion de la hora de salida, llegada y duracion del vuelo

Notar que el xpath comienza con ".", lo que indica que sólo debe buscar en los hijos de ese elemento. Si no ponemos el ".", busca en todo el árbol.<br>
Elementos importantes de xpath:
- "//" busca en todos los hijos del elemento. "/" busca sólo en hijos directos
- "." indica que la búsqueda debe empezar en ese elemento y no en el origen del árbol
- Los atributos de los tags se buscan entre [] y con @

In [5]:
#seleccionamos el primer vuelo
vuelo_1=vuelos[0]
#hora de salida
hora_salida=vuelo_1.find_element('xpath','//div[@class="sc-ixltIz dfdfxH flight-information"]/span[1]').text
print (hora_salida)

10:50


In [6]:
#hora de llegada
hora_llegada=vuelo_1.find_element('xpath','.//div[3]/span[1]').text.replace('\n+1','')
print (hora_llegada)

13:55


In [7]:
# Duracion del vuelo
duracion_vuelo= vuelo_1.find_element('xpath','.//div[2]/span[2]').text
print (duracion_vuelo)

23 h 5 min


veremos cómo obtener la información de las escalas de cada vuelo. <br>
Para desplegar esa información de las escalas, debemos clickear en link para que se habilite el modal que contiene la informacion. Seleccionémoslo:

In [8]:
link_escalas = vuelo_1.find_element('xpath','//div[@class="sc-fYAFcb kdctDt"]//a')
print (link_escalas)

<selenium.webdriver.remote.webelement.WebElement (session="b25c4f0b-2236-45f6-ac4d-42119dd974a4", element="dd0bff41-8f04-457b-b693-30f7bf6e83ea")>


Una vez ubicado el elemento podemos clickear sobre él

In [9]:
link_escalas.click()

Y vemos cómo se despliega la información que estamos buscando enn el driver del navegador. **Notar que cambió el html de la página al hacer click sobre ese botón**

ahora debemos seleccionar los segmentos que contiene el vuelo para calcular la cantidad de escalas, actualmente vemos que las paradas estan contenidas en elementos sections, asi que debemos contabilizar las paradas

In [10]:
segmentos= link_escalas.find_elements('xpath','//div[@class="MuiDialogContent-root sc-fjdhpX jvVjjf"]//section[@class="sc-fEVUGC gIelIH"]')
# print(segmentos, len(segmentos))
for i in segmentos:
    print (i)

<selenium.webdriver.remote.webelement.WebElement (session="b25c4f0b-2236-45f6-ac4d-42119dd974a4", element="94fdcb8b-58f6-4322-af0c-18416da7262a")>
<selenium.webdriver.remote.webelement.WebElement (session="b25c4f0b-2236-45f6-ac4d-42119dd974a4", element="8e30bec8-53a5-40c2-a5e0-473bd8affb2d")>


para calcular las escalas, debemos restar -1 la cantidad total de paradas

In [11]:
#0 escalas si es un vuelo directo
escala=len(segmentos)-1
print(escala)

1


ahora obtengamos desde el modal el Itinerario de vuelo:
- origen ✔
- hora de salida 
- destino vuelo ✔
- hora de llegada
- duración del vuelo ✔
- duración de la escala. *Tip: el último segmento no tendrá esta información*
- número del vuelo
- modelo del avión

In [12]:
for i in segmentos:
    segmento = i
    salida=segmento.find_element('xpath','.//div[@class="sc-dhVevo bQUzMb"]/div[@class="sc-fqCOlO lpqwwl"]/div[@class="iataCode"]/span[1]').text
    hora_salida=segmento.find_element('xpath','.//div[@class="sc-dhVevo bQUzMb"]/div[@class="sc-fqCOlO lpqwwl"]/div[@class="iataCode"]/span[2]').text
    duracion_segmento=segmento.find_element('xpath','.//div[@class="sc-dhVevo bQUzMb"]/div[@class="sc-BOulX kmIWrd"]/span[2]').text
    destino_segmento=segmento.find_element('xpath','.//div[@class="sc-hAcydR ikJhgn"]/div[@class="iataCode"]/span[1]').text
    numero_vuelo_segmento=segmento.find_element('xpath','.//div[@class="sc-hlELIx iUypDF plane-info"]//div[@class="sc-bscRGj iggkUa airline-wrapper"]').text
    modelo_avion_segmento=segmento.find_element('xpath','.//div[@class="sc-hlELIx iUypDF plane-info"]//span[@class="airplane-code"]').text
    
    print(f'Salida:{salida}\nHora Salida:{hora_salida}\nDuracion: {duracion_segmento}\nLlegada:{destino_segmento}\nNumero de vuelo:{numero_vuelo_segmento}\nModelo Avion:{modelo_avion_segmento}\n')


Salida:ASU
Hora Salida:10:50
Duracion: 2 h 50 min
Llegada:SCL
Numero de vuelo:LA1324
Modelo Avion:Airbus A320

Salida:SCL
Hora Salida:21:20
Duracion: 12 h 35 min
Llegada:MAD
Numero de vuelo:LA706
Modelo Avion:Boeing B787-9



In [19]:
duracion_escala_vuelo=link_escalas.find_element('xpath','//section[@class="sc-fEVUGC gAwpmW"]//span[@class="time"]').text
print(f'Duracion Escala:{duracion_escala_vuelo}')

Duracion Escala:7 h 40 min


Una vez que hayamos obtenido toda la información, debemos cerrar el modal/pop-up.

In [23]:
driver.find_element('xpath','//*[@id="itinerary-modal-0-dialog-close"]').click()

Paso 4: cerrar el navegador

In [ ]:
driver.close()